In [1]:
import requests
import json
import os
import numpy as np
from os.path import join
from typing import Tuple, List, Dict, Union, Set
from time import sleep
import pickle

In [2]:
import matplotlib.pyplot as plt 
from tqdm.notebook import tqdm

In [3]:
api_url = 'http://45.14.48.80:10000/'
upload_url = api_url + 'upload'
task_url = api_url + 'task'
healthcheck_url = api_url + 'healthcheck'
results_url = api_url + 'results'

CURL_POST = 'curl -vvvv -X POST'
CURL_GET = 'curl -vvvv -X GET'

cwd = os.getcwd()
models_path = 'data/models'

In [4]:
def upload(full_path: str, upload_url: str) -> Tuple[str, str]:
    r = json.loads(os.popen(f'{CURL_POST} -H "User-Id: test" --form file="@{full_path}" "{upload_url}"').read())
    uuid = r['uuid']
    filename = r['filename']
    return uuid, filename

In [5]:
def task(uuid: str, filename: str, task_url: str):
    config = {
        'uuid' : uuid,
        'verison': '1',
        'file': filename,
        'id': 'test',
        'is_part': False
    }
    config = json.dumps(config)
    os.popen(f'{CURL_POST} -d \'{config}\' "{task_url}"').read()

In [6]:
def healthcheck(healthcheck_url: str) -> Tuple[str, str]:
    r = json.loads(os.popen(f'{CURL_GET} "{healthcheck_url}"').read())
    return r['status'], r['progress']

In [7]:
def get_bins_number_set(results_url: str) -> Set[int]:
    r = json.loads(os.popen(f'{CURL_GET} "{results_url}"').read())['histogram_data']
    return set([el['intervals'] for el in r if not el['type'].startswith('hull')])

In [8]:
def get_hists(bins_num: int, results_url: str) -> Dict[str,  List[float]]:
    r = json.loads(os.popen(f'{CURL_GET} "{results_url}"').read())['histogram_data']
    hists = [el for el in r if (el['intervals'] == bins_num) and not el['type'].startswith('hull')]
    res = {}
    for el in hists: 
        res[el['type']] = el['data']
    
    return res

In [9]:
def get_all_hists(results_url: str) -> Dict[int, Dict[str,  List[float]]]:
    r = json.loads(os.popen(f'{CURL_GET} "{results_url}"').read())['histogram_data']
    res = {}
    for el in r:
        if not el['type'].startswith('hull'):
            res.setdefault(el['intervals'], {})[el['type']] = el['data']
            
    return res

In [10]:
def collect_hists(full_path: str) -> Dict[int, Dict[str, List[float]]]:
    uuid, filename = upload(full_path, upload_url)
    task(uuid, filename, task_url)

    pbar = tqdm(total=100)
    status, progress = healthcheck(healthcheck_url)
    while status != 'COMPLETED':
        status, progress = healthcheck(healthcheck_url)
        pbar.update(round(progress - pbar.n))
        sleep(1)
        
    return get_all_hists(results_url)

In [11]:
models_lst = [el for el in os.listdir(models_path) if el.endswith('.stl')]
models_lst[:2]

['eea8de8c-071c-40a9-b890-51c786838889.stl',
 '4de1f31b-9dc8-4eb7-987a-19caf8ad70ca.stl']

In [12]:
dataset = {}
for model_name in models_lst:
    full_path = f'{cwd}/{models_path}/{model_name}'
    dataset[model_name] = collect_hists(full_path)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/stas/venvs/main/lib/python3.7/site-packages/tqdm/_monitor.py", line 78, in run
    instances = self.get_instances()
  File "/home/stas/venvs/main/lib/python3.7/site-packages/tqdm/_monitor.py", line 58, in get_instances
    return [i for i in self.tqdm_cls._instances.copy()
  File "/usr/lib/python3.7/_weakrefset.py", line 92, in copy
    return self.__class__(self)
  File "/usr/lib/python3.7/_weakrefset.py", line 50, in __init__
    self.update(data)
  File "/usr/lib/python3.7/_weakrefset.py", line 119, in update
    for element in other:
  File "/usr/lib/python3.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [13]:
with open('dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)

In [17]:
with open('dataset.pkl', 'rb') as f:
    print(len(pickle.load(f)))

100
